# Notebook Set-Up

## Install Dependencies

In [ ]:
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import numpy as np
import random
import matplotlib.pyplot as plt
import time
import copy
import os
import csv
import time
import pandas as pd

## Mount Google Drive

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set Hyperperameters

In [42]:
outModelName = 'jess_dev_multilabel_posture' # CHANGE THIS TO SET-UP NEW RESULTS AND CHECKPOINTS FILENAMES
experiment_type = 'train_model' # choose from ['train_model', 'transfer_model]
experiment_name = 'posture_motion'  # train_model options ['social','motion','posture','posture_motion']
                            # transfer_model options ['motion_to_social','posture_to_social','posture_motion_to_social','default_to_social']

batch_size = 64
base_learning_rate = 1e-3
droprate = 0.1
max_epoch = 2
use_dropout = True # add dropout layers (check this is consistent w/ checkpoint model)

best_accuracy = 0
start_epoch = 0  # start from epoch 0

In [43]:
if experiment_type == 'train_model':
  use_checkpoint = False # use default Resnet-50 weights
  freeze_layers = False # gradient descent on all layers
elif experiment_type == 'transfer_model':
  use_checkpoint = True # use pretrained weights from 'checkpoint_t7'
  freeze_layers = True # freeze all but the output layer(s) (fc)

if experiment_name == 'social':
  weights = 'social'
  dataset = 'social_interaction'
  labels = ['social_interaction']
elif experiment_name == 'motion':
  weights = 'motion'
  dataset = 'motion'
  labels = ['motion']
elif experiment_name == 'posture':
  weights = 'posture'
  dataset = 'posture'
  labels = ['posture']
elif experiment_name == 'posture_motion':
  weights = 'posture_motion'
  dataset = 'posture_motion'
  labels = ['posture','motion']
elif experiment_name == 'motion_to_social':
  weights = 'motion'
  dataset = 'social_interaction'
  labels = ['social_interaction']
elif experiment_name == 'posture_to_social':
  weights = 'posture'
  dataset = 'social_interaction'
  labels = ['social_interaction']
elif experiment_name == 'posture_motion_to_social':
  weights = 'posture_motion'
  dataset = 'social_interaction'
  labels = ['social_interaction']
elif experiment_name == 'default_to_social':
  weights = 'n/a'
  dataset = 'social_interaction'
  labels = ['social_interaction']
  use_checkpoint = False # use default Resnet-50 weights

## Set Directories

In [44]:
images_dir = '/content/drive/Shareddrives/NMA-DL/images/final_model/'
result_dir = '/content/drive/Shareddrives/NMA-DL/results/'
annotations_csv = '/content/drive/Shareddrives/NMA-DL/annotations/annotations.csv'

if not os.path.exists(result_dir):
  os.makedirs(result_dir)

# Social Interactions (with Duplicates)
social_interaction_dir = [images_dir + 'train_social_duplicates', images_dir + 'valid_social_duplicates', images_dir + 'test_social_duplicates']
social_interaction_checkpoint = result_dir + 'experiment1/social_interactions/07_25_2022/experiment1_social-interactions_duplicates_batchsize64_lre-3_dropout0.1_epochs400_checkpoint.t7'

# Posture (no Duplicates)
posture_dir = [images_dir + 'train_posture_NOduplicates', images_dir + 'valid_posture_NOduplicates', images_dir + 'test_posture_NOduplicates']
posture_checkpoint = result_dir + 'experiment1/posture/experiment1_posture_NOduplicates_batchsize64_lre-3_dropout0.1_epochs200_checkpoint.t7'

# Motion (no Duplicates)
motion_dir = [images_dir + 'train_motion_NOduplicates', images_dir + 'valid_motion_NOduplicates', images_dir + 'test_motion_NOduplicates']
motion_checkpoint = result_dir + 'experiment1/motion/experiment1_motion_NOduplicates_batchsize64_lre-3_dropout0.1_epochs200_checkpoint.t7'

# Posture + Motion (Multi-Label)
posture_motion_dir = posture_dir
posture_motion_checkpoint = ''

train_dir = eval(dataset + '_dir')[0]
val_dir = eval(dataset + '_dir')[1] 
test_dir = eval(dataset + '_dir')[2]
checkpoint_t7 = eval(weights + '_checkpoint')

# Get Data and Annotations

## Load Datasets using DataLoader

In [45]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        dataset = super(ImageFolderWithPaths, self).__getitem__(index)
        # append the image file path
        return (dataset + (self.imgs[index][0],))

In [46]:
# Transform Properties
data_transforms = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.ColorJitter(brightness=0.5, hue=0.3),
            transforms.RandomGrayscale(p=0.5),
            transforms.RandomRotation(degrees=5),
            transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

# Load Data
train_dataset = ImageFolderWithPaths(train_dir, data_transforms)
val_dataset = ImageFolderWithPaths(val_dir, data_transforms)
test_dataset = ImageFolderWithPaths(test_dir,data_transforms)

# Data Loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Load Annotations and Get Classes

In [47]:
def classDictionary(label):
  '''Return a dictionary where keys = class and values = integer for the label'''
  keys = np.unique(np.array(os.listdir(eval(label + '_dir')[0])))
  keys = [key for key in keys if '.' not in key]
  values = np.arange(len(keys))
  return dict(zip(keys, values))

def getLabelsFromPaths(paths, labels=['posture','motion','social_interaction']):
  '''Return a dictionary of keys equal to the input labels and values equal to 
  the index of the classes in that label'''
  
  classes = {}
  exclude_inputs = []
  for label in labels:

    # Get actions and classes corresponding to the label 
    labelAnnotations = annotations[annotations['VerbType'] == label]
    labelClasses = eval('classes_' + label)
    
    values = []
    for path in paths:
      fileName = os.path.basename(path)

      # Get actions corresponding to the label and file
      actions = list(labelAnnotations[labelAnnotations['fileName'] == fileName]['Action'])

      # Remove actions not in label classes
      actions = [action for action in actions if action in labelClasses]

      # If no action found, don't use that image. If more than one 
      # action found, choose first value
      if len(actions)==0:
        action = -1
      else:
        action = labelClasses[actions[0]] # CAN CHANGE TO RANDOM CHOICE IF DUPLICATES
      
      values.append(action)
      
    # Exclude inputs with no class
    exclude_inputs = exclude_inputs + [index for (index, item) in enumerate(values) if item == -1]
    classes[label] = torch.tensor(values)

  exclude_inputs = list(np.unique(exclude_inputs))
  include_inputs = [ind for ind in range(len(paths)) if ind not in exclude_inputs]
  return classes, include_inputs

In [48]:
# Load Annotations
annotations = pd.read_csv(annotations_csv)
annotations['Action'] = annotations['Action'].str.replace('/','-') # Fix jump/fall

# Get Classes for Each Label
classes_motion = classDictionary('motion')
classes_posture = classDictionary('posture')
classes_social_interaction = classDictionary('social_interaction')

# Create Label Dictionary
labelDictionary = {}
for label in labels:
  labelDictionary[label] = len(eval('classes_' + label))
labelDictionary

print(f'Motion Classes ({len(classes_motion)}): {list(classes_motion.keys())}')
print(f'Posture Classes ({len(classes_posture)}): {list(classes_posture.keys())}')
print(f'Social Classes ({len(classes_social_interaction)}): {list(classes_social_interaction.keys())}')

Motion Classes (6): ['board', 'jump-fall', 'ride', 'run', 'still', 'walk']
Posture Classes (5): ['bend', 'crouch', 'lay', 'sit', 'stand']
Social Classes (13): ['act_on', 'carry', 'fistbump', 'handshake', 'highfive', 'hold_sb', 'hug', 'kiss', 'pat', 'point_sb', 'pull_sb_soft', 'thumbsup', 'wave']


# Define Model

## Define Custom Output and Dropout Layers

In [49]:
# Get names of model layers with the exception of output layers
global layer_names
layer_names = []
for name, module in models.resnet50().named_children():
  layer_names.append(name)
layer_names = layer_names[:-1]

In [50]:
class resnetCustomOutput(nn.Module):
    def __init__(self, net, labelDictionary):
        super().__init__()

        # Get input features from last layer
        in_features = getattr(net,list(net.named_children())[-1][0]).in_features

        # Assign all layers but fc to new model
        for name, module in net.named_children():
          if 'fc' not in name:
            setattr(self, name, module)
        
        # Add new output layer(s) using labelDictionary {'label': num_classes}
        for label in labelDictionary:
          outputLayer = nn.Linear(in_features=in_features, out_features=labelDictionary[label])
          setattr(self, label, outputLayer)

    def forward(self, x):
        # Forward pass of all network layers except the output layer(s)
        for name in layer_names:
          x = getattr(self, name)(x)
        
        x = torch.flatten(x, 1)
        
        # Get fully-connected outputs for each label in the model
        output = { }
        for label in labelDictionary:
          output[label] = getattr(self, label)(x)

        return output

In [51]:
def append_dropout(model, rate = droprate):
  '''Add dropout layer after each nn.Conv2d layer'''
  for name, module in model.named_children():
    if len(list(module.children())) > 0:
      append_dropout(module)
    if isinstance(module, nn.Conv2d):
      new = nn.Sequential(module, nn.Dropout2d(p=rate, inplace=True))
      setattr(model, name, new)

In [52]:
def getLabelDictionaryFromCheckpoint(checkpoint):
  '''Looks at checkpoint file, compares to model layers and returns a
  labelDictionary {'label': num_classes} matching the checkpoint file'''

  # Get output layer names
  checkpoint_output = []
  for key in list(checkpoint['state_dict'].keys()):
    checkpoint_key = key.split('.', 1)[0]
    if checkpoint_key not in layer_names:
      checkpoint_output.append(checkpoint_key)
  checkpoint_output = list(np.unique(checkpoint_output))

  # Get output layer classes
  checkpoint_labelDictionary = {}
  for label in checkpoint_output:
    checkpoint_labelDictionary[label] = checkpoint['state_dict'][checkpoint_output[0] + '.weight'].shape[0]
  return checkpoint_labelDictionary

## Define RESNET-50 with options (i.e. dropout, checkpoints, num_classes, freeze, multilabel)

In [53]:
# Use pytorch models using pre-trained weights
net = models.resnet50(pretrained=True)

# Add dropout layers after each Conv2d
if use_dropout:
  append_dropout(net)

# If using pretrained weights from checkpoint file, load them
if use_checkpoint:
  checkpoint = torch.load(checkpoint_t7)

  # Get labelDictionary {'label': num_classes} from state_dict()
  checkpoint_labelDictionary = getLabelDictionaryFromCheckpoint(checkpoint)

  # Change model outputs to match pretrained model
  net = resnetCustomOutput(net, checkpoint_labelDictionary)

  # Load checkpoints
  net.load_state_dict(checkpoint['state_dict'])

# Change model outputs to match new model inputs
net = resnetCustomOutput(net, labelDictionary)

# If freezing, freeze all but the output layer weights/biases
for name, param in net.named_parameters():
  if freeze_layers:
    if any(label in name for label in labelDictionary.keys()):
      param.requires_grad = True
    else:
      param.requires_grad = False
  else:
    param.requires_grad = True

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Define Loss and Optimization Functions

In [54]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=base_learning_rate, momentum=0.9, weight_decay=1e-4)

## Define Training and Test Functions

In [55]:
from sys import path_hooks
def train(net, epoch, labels):
  '''Train the model on the test set'''
  print('\nEpoch: %d' % epoch)
  net.train()

  train_loss = dict()
  correct = dict()
  total = dict()
  for label in labels:
    train_loss[label] = 0
    correct[label] = 0
    total[label] = 0

  for batch_idx, (inputs, _, paths) in enumerate(train_dataloader):
    targets, include_inputs = getLabelsFromPaths(paths, labels)

    optimizer.zero_grad()

    inputs = Variable(inputs[include_inputs,:,:,:])
    for key in targets:
      targets[key] = Variable(targets[key][include_inputs])

    outputs = net(inputs)

    loss = 0
    for key in targets:
      loss += criterion(outputs[key], targets[key])
    
    loss.backward()
    optimizer.step()

    for key in targets:
      train_loss[key] += loss.item()
      _, predicted = torch.max(outputs[key].data, 1)
      total[key] += targets[key].size(0)
      correct[key] += predicted.eq(targets[key].data).cpu().sum()

      if batch_idx % 500 == 0:
        print(batch_idx, len(train_dataloader), key, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss[key]/(batch_idx+1), 100.*correct[key]/total[key], correct[key], total[key]))

  accuracy = dict()
  for key in targets:
    accuracy[key] = 100.*correct[key]/total[key]
    accuracy[key] = accuracy[key].item()
    train_loss[key] = train_loss[key]/batch_idx
  # return (train_loss/batch_idx, 100.*correct/total)
  return (train_loss, accuracy, total)

In [56]:
def test(net, epoch, labels, outModelName):
  '''Test the model on the validation set'''
  global best_accuracy
  net.eval()

  test_loss = dict()
  correct = dict()
  total = dict()
  for label in labels:
    test_loss[label] = 0
    correct[label] = 0
    total[label] = 0

  with torch.no_grad():
    for batch_idx, (inputs, _, paths) in enumerate(val_dataloader):
      targets, include_inputs = getLabelsFromPaths(paths, labels)
      
      inputs = Variable(inputs[include_inputs,:,:,:])
      for key in targets:
        targets[key] = Variable(targets[key][include_inputs])

      outputs = net(inputs)

      loss = 0
      for key in targets:
        loss += criterion(outputs[key], targets[key])

      for key in targets:
        test_loss[key] += loss.item()
        _, predicted = torch.max(outputs[key].data, 1)
        total[key] += targets[key].size(0)
        correct[key] += predicted.eq(targets[key].data).cpu().sum()

        if batch_idx % 200 == 0:
          print(batch_idx, len(val_dataloader), key, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
              % (test_loss[key]/(batch_idx+1), 100.*correct[key]/total[key], correct[key], total[key]))

  # Save checkpoint
  key = list(outputs.keys())[0]
  accuracy = 100.*correct[key]/total[key]
  if accuracy > best_accuracy:
    best_accuracy = accuracy
    checkpoint(net, accuracy, epoch, outModelName)

  accuracy = dict()
  for key in targets:
    accuracy[key] = 100.*correct[key]/total[key]
    accuracy[key] = accuracy[key].item()
    test_loss[key] = test_loss[key]/batch_idx

  # return (test_loss/batch_idx, 100.*correct/total)
  return (test_loss, accuracy, total)

## Define Checkpoint and Learning Rate Adjustment

In [57]:
checkpointName = result_dir + outModelName + '_checkpoint.t7'

def checkpoint(model, acc, epoch, outModelName):
  '''Save check point'''
  print('Saving..')
  state = {
      'state_dict': model.state_dict(),
      'acc': acc,
      'epoch': epoch,
      'rng_state': torch.get_rng_state()
  }
  torch.save(state, checkpointName)

def adjust_learning_rate(optimizer, epoch):
  '''Decrease the learning rate at 100 and 150 epoch'''
  lr = base_learning_rate
  # if epoch >= 100:
  #   lr /= 10
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr

# Train Model

In [ ]:
logName = result_dir + outModelName + '_result.csv'
startTime = time.time()

if not os.path.exists(logName):
  with open(logName, 'w') as logfile:
      logwriter = csv.writer(logfile, delimiter=',')
      logwriter.writerow(['epoch', 'train_loss', 'train_acc', 'test_loss', 'test_acc','time_elapsed','train_batch','test_batch'])

for epoch in range(start_epoch, max_epoch):
  adjust_learning_rate(optimizer, epoch)
  train_loss, train_acc, train_batch = train(net, epoch, labels)
  test_loss, test_acc, test_batch = test(net, epoch, labels, outModelName)
  
  with open(logName, 'a') as logfile:
    logwriter = csv.writer(logfile, delimiter=',')
    time_elapsed = time.time() - startTime
    logwriter.writerow([epoch, train_loss, train_acc, test_loss, test_acc, time_elapsed, train_batch, test_batch])
  print(f'Epoch: {epoch} | train acc: {train_acc} | test acc: {test_acc} | time: {time_elapsed}')


Epoch: 0
0 35 posture Loss: 3.574 | Acc: 16.129% (10/62)
0 35 motion Loss: 3.574 | Acc: 3.226% (2/62)


In [ ]:
train_loss

In [ ]:
train_acc

In [ ]:
test_loss

In [ ]:
test_acc